In [1]:
import numpy as np
import copy as cp
import csv

In [71]:
class Sudoku:
    
    board = None
    size = None
    
    def __init__(self,file = None, board = None, size = None):
        
        if file:
            self.board = np.zeros((9,9))
            self.size = 9
            with open(file) as csv_file:
                csv_reader = csv.reader(csv_file,delimiter = ',')
                
                for i,row in enumerate(csv_reader):
                    for j,number in enumerate(row[1:]):
                        if number != '':
                            self.board[i,j] = number
        else:
            self.board = board
            self.size = size


    def printBoard(self):
        
        for i in range(0,9):
            print self.board[i,0:3], "|",self.board[i,3:6],"|",self.board[i,6:9]
            if (i+1)%3 == 0 and i!= 8:
                print "------------------------------------"
        
    def getRange(self,cuadrant):
        
        row,col = [],[]
        
        if cuadrant <3 :
            row = range(0,3)
            
        if cuadrant >= 3 and cuadrant< 6:
            row = range(3,6)
            
        if cuadrant >= 6 and cuadrant < 9:
            row = range(6,9)

        if cuadrant in (0,3,6):
            col = range(0,3)
        
        if cuadrant in (1,4,7):
            col = range(3,6)
                    
        if cuadrant in (2,5,8):
            col = range(6,9)
        
        return row,col
        
    def getCuadrant(self,pos):
        
        row,col =  pos
        
        if row <3:
            cuad = 0
        
        if row >=3 and row <6:
            cuad = 3
        
        if row >=6:
            cuad = 6
        
        cuad += col/3
        
        return cuad
        
        
    
    def checkCuadrant(self,pos):
        cuad = self.getCuadrant(pos)        
        rows,cols = self.getRange(cuad)
        
        if self.board[pos] == 0:
            return True
            
        
        for i in rows:
            for j in cols:
                if (i,j) != pos and self.board[i,j] == self.board[pos]:
                    return False
        
        return True
    
    def getRowsandCols(self,pos):
        row,col = pos
        moves = []
        for i in range(self.size):
            if i != row and not (i,col) in moves:
                moves.append((i,col))
            if i != col and not (row,i) in moves:
                moves.append((row,i))
        
        return moves
                
            
    
    def checkRowandCol(self,pos):
        row,col = pos
        
        if self.board[pos]==0:
            return True
        
        moves = self.getRowsandCols(pos)
        
        for mov in moves:
            if self.board[mov] == self.board[row,col]:
                return False            
        return True    
    
    #Funcion que verifica que la posicion no tenga conflictos
    def isOk(self,pos):
        
        if not self.checkCuadrant(pos) or not self.checkRowandCol(pos):
            return False
        return True
    
    def isValid(self):

        for i in range(self.size):
            for j in range(self.size):
                if not self.isOk((i,j)):
                    return False
        return True
    
    #Funcion que verifica que el tablero este resuelto
    def isGoal(self): 
     
        for i in range(self.size):
            for j in range(self.size):
                if not self.isOk((i,j)) or self.board[i,j] == 0:
                    return False
        return True

In [81]:
board = Sudoku("solver.csv")
#board = Sudoku("Sudoku.csv")
board.printBoard()


#board.isValid()
# 1,4,7  col (3,6)
# 2,5,8  col (6,9)
# 0 ,3,6 col (0,3)

# < 3 ro

[8. 1. 2.] | [7. 9. 3.] | [5. 4. 6.]
[4. 9. 7.] | [1. 6. 5.] | [3. 8. 2.]
[3. 6. 5.] | [2. 8. 4.] | [1. 9. 7.]
------------------------------------
[2. 8. 4.] | [5. 3. 7.] | [9. 6. 1.]
[1. 5. 9.] | [6. 2. 8.] | [7. 3. 4.]
[7. 3. 6.] | [9. 4. 1.] | [8. 2. 5.]
------------------------------------
[6. 7. 3.] | [4. 5. 9.] | [2. 1. 8.]
[9. 4. 1.] | [8. 7. 2.] | [6. 5. 3.]
[5. 2. 8.] | [3. 1. 6.] | [4. 7. 9.]


In [158]:
class Back(Sudoku):
    board = None
    neighbours = None
    
    def __init__(self,file = None, board = None, size = None):
        
        Sudoku.__init__(self,file,board,size)
        self.getNeighbours()
    
    def getEmpty(self):
        empty = []
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i,j] == 0:
                    empty.append((i,j))
        return empty
    
    def createNeighbour(self,pos,value):
        
        board = self.board.copy()
        board[pos] = value
        return (board,self.size)
        
    
    def getNeighbours(self):
        self.neighbours = []
        values = range(1,10)
        
        empty = self.getEmpty()
        for i in empty:
            for val in values:
                self.neighbours.append(self.createNeighbour(i,val))

file = "solver.csv"
#file = "Sudoku.csv"
backtracking = Back(file)
backtracking.printBoard()

[8. 1. 2.] | [7. 9. 3.] | [5. 4. 6.]
[4. 9. 7.] | [1. 6. 5.] | [3. 8. 2.]
[3. 6. 5.] | [2. 8. 4.] | [1. 9. 7.]
------------------------------------
[2. 8. 4.] | [5. 3. 7.] | [9. 6. 1.]
[1. 5. 9.] | [6. 2. 8.] | [7. 3. 4.]
[7. 3. 6.] | [9. 4. 1.] | [8. 2. 5.]
------------------------------------
[6. 7. 3.] | [4. 5. 9.] | [2. 1. 8.]
[9. 4. 1.] | [8. 7. 2.] | [6. 5. 3.]
[5. 2. 8.] | [3. 1. 6.] | [4. 7. 9.]


In [87]:
def BackTracking(node):
    
    if not node.isValid():
        return False
    
    if node.isGoal():
        return node
    
    for vec in node.neighbours:

        ans =  BackTracking(Back(None,vec[0],vec[1]))
        if ans:
            return ans
    
    return False
    
    
#ans = BackTracking(backtracking)
ans.printBoard()

[8. 1. 2.] | [7. 9. 3.] | [5. 4. 6.]
[4. 9. 7.] | [1. 6. 5.] | [3. 8. 2.]
[3. 6. 5.] | [2. 8. 4.] | [1. 9. 7.]
------------------------------------
[2. 8. 4.] | [5. 3. 7.] | [9. 6. 1.]
[1. 5. 9.] | [6. 2. 8.] | [7. 3. 4.]
[7. 3. 6.] | [9. 4. 1.] | [8. 2. 5.]
------------------------------------
[6. 7. 3.] | [4. 5. 9.] | [2. 1. 8.]
[9. 4. 1.] | [8. 7. 2.] | [6. 5. 3.]
[5. 2. 8.] | [3. 1. 6.] | [4. 7. 9.]


In [143]:
class Ac3(Sudoku):
    board = None
    domains = None
    constraints = None
    variables = None
    asig = None
    def __init__(self,file = None, board = None, size = None):
        
        Sudoku.__init__(self,file,board,size)
        
        self.variables = []
        self.domains = {}
        self.constraints = {}
        self.asig = {}
        
        #Creamos un arreglo con las variables
        for i in range(self.size):
            for j in range(self.size):
                self.variables.append((i,j))
                
        #Creamos un arreglo con los contraints para cada variable
        
        for var in self.variables:
            self.constraints[var] = []
        
        #Obtenemos los constraints de acuerdo a los valores iniciales
        for i in range(self.size):
            for j in range(self.size):
                
                if self.board[i,j] != 0:
                    
                    self.asig[i,j] = self.board[i,j]
                    moves = self.getRowsandCols((i,j))
                    value = self.board[i,j]
                    
                    for mv in moves:
                        if not value in self.constraints[mv]:

                            self.constraints[mv].append(value)
        
        for var in self.variables:
            
            posdomain = range(1,self.size+1)
            self.domains[var] = []
            
            for i in posdomain:
                if not i in self.constraints[var]:
                    self.domains[var].append(i)
                    
    
    
    
p = Ac3("Sudoku.csv")
p.printBoard()


[0. 1. 0.] | [7. 9. 0.] | [5. 4. 0.]
[0. 0. 0.] | [0. 6. 0.] | [0. 0. 0.]
[0. 6. 0.] | [0. 0. 4.] | [0. 9. 0.]
------------------------------------
[2. 0. 0.] | [5. 0. 0.] | [0. 0. 1.]
[0. 0. 9.] | [0. 0. 0.] | [0. 0. 0.]
[7. 0. 0.] | [0. 0. 0.] | [8. 2. 5.]
------------------------------------
[0. 0. 0.] | [0. 0. 0.] | [0. 1. 0.]
[0. 4. 1.] | [0. 0. 2.] | [6. 0. 0.]
[5. 0. 0.] | [3. 0. 0.] | [0. 0. 0.]


In [160]:
def arc_reduce(node,var1,var2):

    flag = False
    
    for i in node.domains[var1]:
        
        if i in node.domains[var2]:
            
            index = node.domains[var1].index(i)
            node.domains[var1].pop(index)
            
            
            flag = True

    return flag

p = Ac3("Sudoku.csv")
print p.domains[0,0]
for i in p.getRowsandCols((0,0)):
    print p.constraints[i]
    if arc_reduce(p,(0,0),i):
        print "redu"
    
print p.domains[0,0]

[3, 6, 8]
[6.0, 2.0, 7.0, 5.0]
redu
[7.0, 9.0, 5.0, 4.0, 6.0]
[6.0, 4.0, 9.0, 2.0, 7.0, 5.0]
[1.0, 7.0, 9.0, 5.0, 4.0]
redu
[5.0, 1.0, 7.0]
[1.0, 9.0, 5.0, 4.0, 3.0]
[2.0, 9.0, 7.0, 5.0]
[1.0, 7.0, 5.0, 4.0, 6.0]
[2.0, 8.0, 5.0]
[1.0, 7.0, 9.0, 5.0, 4.0, 2.0]
[2.0, 7.0, 1.0, 5.0]
[1.0, 7.0, 9.0, 4.0, 8.0, 6.0]
[2.0, 7.0, 4.0, 1.0, 6.0, 5.0]
[1.0, 7.0, 9.0, 5.0, 2.0]
[2.0, 7.0, 3.0]
[1.0, 7.0, 9.0, 5.0, 4.0]
[]


In [117]:
def AC3(node):
    
    for var in node.variables:
        
        print var



#AC3(p)

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(2, 0)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(2, 5)
(2, 6)
(2, 7)
(2, 8)
(3, 0)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(3, 5)
(3, 6)
(3, 7)
(3, 8)
(4, 0)
(4, 1)
(4, 2)
(4, 3)
(4, 4)
(4, 5)
(4, 6)
(4, 7)
(4, 8)
(5, 0)
(5, 1)
(5, 2)
(5, 3)
(5, 4)
(5, 5)
(5, 6)
(5, 7)
(5, 8)
(6, 0)
(6, 1)
(6, 2)
(6, 3)
(6, 4)
(6, 5)
(6, 6)
(6, 7)
(6, 8)
(7, 0)
(7, 1)
(7, 2)
(7, 3)
(7, 4)
(7, 5)
(7, 6)
(7, 7)
(7, 8)
(8, 0)
(8, 1)
(8, 2)
(8, 3)
(8, 4)
(8, 5)
(8, 6)
(8, 7)
(8, 8)
